In [1]:
%load_ext autoreload
%autoreload 2


In [ ]:
import pandas as pd
from preprocessor import load_cleaned_data, select_and_map_column_for_recai, create_inter_file

import os

In [3]:
folder_path = "../cleaned_data"
folder_save = "../RecAI/RecLM-eval/data"
list_data =  ["ml-1m", "jobrec", "lfm-1b"]

In [4]:
map_col_name = {
                "movie_title": "title",
                "Title": "title",
                "name": "title",
                    }

In [ ]:
for data in list_data:
    save_path = f"{folder_save}/{data}"

    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    df_train, df_val, df_test, df_user, df_item = load_cleaned_data(f"{folder_path}/{data}")

    #save user attribute of test users only
    df_user = df_user[df_user.in_test]
    list_user_in_test = df_user.user_id

    df_train = df_train[df_train.user_id.isin(list_user_in_test)]
    df_val = df_val[df_val.user_id.isin(list_user_in_test)]
    df_test = df_test[df_test.user_id.isin(list_user_in_test)]

    # print(len(df_train), len(df_val), len(df_test))

    df_train = df_train[df_train.item_id.isin(df_item.item_id)]
    df_val = df_val[df_val.item_id.isin(df_item.item_id)]
    df_test = df_test[df_test.item_id.isin(df_item.item_id)]
    # print(len(df_train), len(df_val), len(df_test))

    #add test user_ids to val
    augmented = df_test[["user_id"]][~df_test.user_id.isin(df_val.user_id)].drop_duplicates().reset_index(drop=True)
    augmented["item_id"] = -1
    augmented["timestamp"] = -1
    augmented["num_repeat"] = -1

    df_val = pd.concat([df_val, augmented])

    df_train_filtered = create_inter_file(df_train, "train", save_path)
    df_val_filtered = create_inter_file(df_val, "val", save_path)
    create_inter_file(df_test, "test", save_path)

    #save the test items, and then the last 10 of train and last 10 of val items of users in test
    set_item = set()

    for row in df_train_filtered.item_id.values:
        list_row = [int(x) for x in row]
        set_item.update(set(list_row))

    for row in df_val_filtered.item_id.values:
        list_row = [int(x) for x in row]
        set_item.update(set(list_row))

    for row in df_test.item_id.values:
        set_item.update(set([row]))

    df_item = df_item[df_item.item_id.isin(set_item)]

    df_item\
        .rename(columns=map_col_name)\
        .to_json(f"{save_path}/metadata.json", index=False, orient="records", lines=True)
    
    df_user = select_and_map_column_for_recai(df_user, data)
    display(df_user)

    df_user.to_json(f"{save_path}/user_data.json", index=False, orient="records", lines=True)